# Model Description
- Apply a Sequence to Sequence model with a Encoder RNN and a Decoder RNN. Parameters related to attention are learned and weighted input hidden states are summed for a final hidden state input for each step of decoding
- train on Uniref59=0_01

## Math and model formulation rerference:
- https://arxiv.org/abs/1506.03134
- https://arxiv.org/pdf/1409.0473.pdf
- https://jalammar.github.io/visualizing-neural-machine-translation-mechanics-of-seq2seq-models-with-attention/

## code reference:
- https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/seq2seq-translation.ipynb
- https://github.com/omarsar/pytorch_neural_machine_translation_attention/blob/master/NMT_in_PyTorch.ipynb

In [ ]:
import torch.nn as nn
import argparse
import random
import warnings
import numpy as np
import torch
import torch.nn.functional as F
from torch import optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.autograd import Variable
import itertools
import pandas as pd
# seed = 7
# torch.manual_seed(seed)
# np.random.seed(seed)

In [ ]:
# pfamA_motors = pd.read_csv("../data/pfamA_motors.csv")
# df_dev = pd.read_csv("../data/df_dev.csv")
# pfamA_motors = pfamA_motors.iloc[:,1:]
# clan_train_dat = pfamA_motors.groupby("clan").head(4000)
# clan_train_dat = clan_train_dat.sample(frac=1).reset_index(drop=True)
# clan_test_dat = pfamA_motors.loc[~pfamA_motors["id"].isin(clan_train_dat["id"]),:].groupby("clan").head(400)
uniref50_01 = pd.read_csv("../data/uniref50_01.tsv",sep = "\t",header=None)
uniref50_01.shape

In [ ]:
clan_train_dat.shape

In [ ]:
def df_to_tup(dat):
    data = []
    for i in range(dat.shape[0]):
        row = dat.iloc[i,:]
        tup = (row["seq"],row["clan"])
        data.append(tup)
    return data

In [ ]:
clan_training_data = df_to_tup(clan_train_dat)
clan_test_data = df_to_tup(clan_test_dat)
for seq,clan in clan_training_data:
    print(seq)
    print(clan)
    break

In [ ]:
aminoacid_list = [
    'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
    'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'
]
clan_list = ["actin_like","tubulin_c","tubulin_binding","p_loop_gtpase"]
        
aa_to_ix = dict(zip(aminoacid_list, np.arange(1, 21)))
clan_to_ix = dict(zip(clan_list, np.arange(0, 4)))

def word_to_index(seq,to_ix):
    "Returns a list of indices (integers) from a list of words."
    return [to_ix.get(word, 0) for word in seq]

ix_to_aa = dict(zip(np.arange(1, 21), aminoacid_list))
ix_to_clan = dict(zip(np.arange(0, 4), clan_list))

def index_to_word(ixs,ix_to): 
    "Returns a list of words, given a list of their corresponding indices."
    return [ix_to.get(ix, 'X') for ix in ixs]

def prepare_sequence(seq):
    idxs = word_to_index(seq[0:-1],aa_to_ix)
    return torch.tensor(idxs, dtype=torch.long)

def prepare_labels(seq):
    idxs = word_to_index(seq[1:],aa_to_ix)
    return torch.tensor(idxs, dtype=torch.long)

In [ ]:
prepare_labels('YCHXXXXX')

In [ ]:
# set device
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, output_size, batch_first=False, bidirectional=True):
        super(Encoder, self).__init__()
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.log_softmax = nn.LogSoftmax(dim= 1)
        self.batch_first = batch_first
        self.aa_embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers,
                           batch_first=batch_first, bidirectional=bidirectional)

    def forward(self, seq):
        # embed each aa to the embedded space
        embedding_tensor = self.aa_embedding(seq)
        #output of shape (seq_len, batch, num_directions * hidden_size):
        outputs, hidden = self.rnn(embedding_tensor.view(len(seq), 1, -1))
        # Return output and final hidden state
        return outputs, hidden

class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        # both forward and backward direction
        self.enc_units = hidden_size*2
        self.dec_units = hidden_size*2
        
        self.W1 = nn.Linear(self.enc_units, self.dec_units, bias=False)
        self.W2 = nn.Linear(self.enc_units, self.dec_units, bias=False)
        self.vt = nn.Linear(self.enc_units, 1, bias=False)

    def forward(self,encoder_outputs,decoder_state):
        
        
        # encoder_outputs: (seq_len, batch_size, hidden_size*2)
        encoder_transform = self.W1(encoder_outputs)
#         print("encoder_transform.shape: ", encoder_transform.shape)

        # (1 (unsqueezed),batch_size, hidden_size*2)
        decoder_transform = self.W2(decoder_state)
#         print("decoder_transform: ", decoder_transform.shape)
        
        combined_transform = encoder_transform + decoder_transform
#         print("combined_transform.shape ", combined_transform.shape)
        # 1st line of Eq.(3) in the paper
        # (seq_len, batch_size = 1 , 1) => squeeze to (seq_len, batch_size)
        u_i = self.vt(torch.tanh(combined_transform)).squeeze()
#         print("u_i.shape ", u_i.shape)

        # log-softmax for a better numerical stability
        attention_weights = F.log_softmax(u_i, dim=0).view(-1, 1, 1)
#         print("attention_weights.shape ", attention_weights.shape)
        
        #context_vector shape after sum == (batch,hidden*2)
        context_vector = attention_weights * encoder_outputs
        context_vector = torch.sum(context_vector, dim=0)
#         print("context_vector.shape ", context_vector.shape)
        return context_vector,attention_weights

    

class Decoder(nn.Module):
    # hidden size refers to input hidden size, decoder hidden size should be 2*encoder hidden size since the decoder is unidirectional
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, output_size, batch_first=False, bidirectional=False):
        super(Decoder, self).__init__()
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.log_softmax = nn.LogSoftmax(dim= 1)
        self.batch_first = batch_first
        self.context_size = hidden_size*2
        self.aa_embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTMCell(input_size=embedding_size + self.context_size, hidden_size=hidden_size*2)
        self.attn = Attention(hidden_size)
        self.fc = nn.Linear(hidden_size*2, output_size)
        
    def forward(self, y, encoder_outputs,decoder_state,decoder_cell):
        
        context_vector,attention_weights = self.attn(encoder_outputs,decoder_state)
#         print("context_vector.shape: ", context_vector.shape)
#         print("attention_weights.shape: ", attention_weights.shape)
        y_embedded = self.aa_embedding(y)
#         print("y_embedded.shape: ", y_embedded.shape)
        y_cat = torch.cat((context_vector.unsqueeze(1), y_embedded), -1)
#         print("y_cat.shape: ", y_cat.shape)
        hidden = self.rnn(y_cat.squeeze(1),(decoder_state,decoder_cell))
        h_i,_ = hidden
#         print("h_i.shape: ", h_i.shape)
        decoded_space = self.fc(h_i)
#         print("decoded_space.shape: ", decoded_space.shape)
        decoded_scores = F.log_softmax(decoded_space, dim = 1)
#         print("decoded_scores.shape: ", decoded_scores.shape)
        decoded_aa = torch.argmax(decoded_scores)
#         print("decoded_aa.shape: ", decoded_aa.shape)
        return decoded_scores, decoded_aa, hidden, attention_weights


In [ ]:
# Hyperparameters
input_size = len(aminoacid_list) + 1
num_layers = 1
hidden_size = 64
output_size = len(aminoacid_list) + 1
embedding_size= 10
learning_rate = 0.001

In [ ]:
encoder = Encoder(input_size = input_size, \
                               embedding_size = embedding_size, \
                               hidden_size = hidden_size, \
                               num_layers = num_layers, \
                               output_size = output_size)
decoder = Decoder(input_size = input_size, \
                               embedding_size = embedding_size, \
                               hidden_size = hidden_size, \
                               num_layers = num_layers, \
                               output_size = output_size)

encoder.to(device)
decoder.to(device)

for m in encoder.modules():
    if isinstance(m, nn.Linear):
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)

for m in decoder.modules():
    if isinstance(m, nn.Linear):
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)


criterion = nn.NLLLoss()                
optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), 
                       lr= learning_rate)

In [ ]:
print(encoder)

In [ ]:
print(decoder)

## Trail test of Model by running 1 epoch

In [ ]:
# loss_vector = []
running_loss = 0
print_every = 100

for epoch in np.arange(0, uniref50_01.shape[0]): 
    seq = uniref50_01.iloc[epoch, 1]
    if(len(seq)>4000):
        continue
    # Step 1. Remember that Pytorch accumulates gradients.
    # We need to clear them out before each instance
    print(epoch)
    print("len(seq): ", len(seq))
    # Step 2. Get our inputs ready for the network, that is, turn them into
    # Tensors of word indices.
    sentence_in = prepare_sequence(seq)
    targets = prepare_labels(seq)
    sentence_in = sentence_in.to(device = device)
    targets = targets.view(1,-1).to(device = device)
    print("targets.shape: ", targets.shape)
    
    print(targets.shape)
    print(sentence_in.shape)
    
    encoder.train()
    decoder.train()
    optimizer.zero_grad()
    
    loss = 0

    seq_len = sentence_in.size(0)

    encoder_outputs, encoder_hidden = encoder(sentence_in)
    print("encoder_outputs.shape: " , encoder_outputs.shape)
    
    encoder_h_n, encoder_c_n = encoder_hidden
    print("encoder_h_n.shape: ", encoder_h_n.shape)
    print("encoder_hidden_last: ", encoder_c_n.shape)
    
    # Lets use zeros as an intial input
    y_0 = 0
    # using zeros for initial decoder hidden and cell state 
    d_0 = Variable(torch.zeros(1, decoder.hidden_size*2))
    dcell_0 = Variable(torch.zeros(1,  decoder.hidden_size*2))
    print("d_0.shape: ", d_0.shape)
    print("dcell_0.shape: ", dcell_0.shape)
    
    y_last = y_0
    d_last, d_cell_last = d_0, dcell_0

    
    print("seq_len: ",seq_len)
    for di in range(seq_len):
        decoded_scores, y_last, (d_last,d_cell_last), attention_weights = decoder(Variable(torch.LongTensor([[y_last]])).to(device), \
                                                                                 encoder_outputs.to(device), \
                                                                                 d_last.to(device),\
                                                                                 d_cell_last.to(device))
        
        loss += criterion(decoded_scores.to(device), targets[:,di])   
    
    assert not np.isnan(loss.item()), 'Model diverged with loss = NaN'

    loss.backward()
    optimizer.step()
    
    if epoch % print_every == 0:
        print(f"At Epoch: %.1f"% epoch)
        print(f"Loss %.4f"% (loss/seq_len))
#     loss_vector.append(loss/seq_len)
    break
    
    

## Train Model on Uniref50_01

In [ ]:
# loss_vector = []
# running_loss = 0
print_every = 1000

for epoch in np.arange(0, uniref50_01.shape[0]): 
    seq = uniref50_01.iloc[epoch, 1]
    if(len(seq)>4000):
        continue
    # Step 1. Remember that Pytorch accumulates gradients.
    # We need to clear them out before each instance
#     print(epoch)
#     print("len(seq): ", len(seq))
    # Step 2. Get our inputs ready for the network, that is, turn them into
    # Tensors of word indices.
    sentence_in = prepare_sequence(seq)
    targets = prepare_labels(seq)
    sentence_in = sentence_in.to(device = device)
    targets = targets.view(1,-1).to(device = device)
#     print("targets.shape: ", targets.shape)
    
#     print(targets.shape)
#     print(sentence_in.shape)
    
    encoder.train()
    decoder.train()
    optimizer.zero_grad()
    
    loss = 0

    seq_len = sentence_in.size(0)

    encoder_outputs, encoder_hidden = encoder(sentence_in)
#     print("encoder_outputs.shape: " , encoder_outputs.shape)
    
    encoder_h_n, encoder_c_n = encoder_hidden
#     print("encoder_h_n.shape: ", encoder_h_n.shape)
#     print("encoder_hidden_last: ", encoder_c_n.shape)
    
    # Lets use zeros as an intial input
    y_0 = 0
    # using zeros for initial decoder hidden and cell state 
    d_0 = Variable(torch.zeros(1, decoder.hidden_size*2))
    dcell_0 = Variable(torch.zeros(1,  decoder.hidden_size*2))
#     print("d_0.shape: ", d_0.shape)
#     print("dcell_0.shape: ", dcell_0.shape)
    
    y_last = y_0
    d_last, d_cell_last = d_0, dcell_0

    
#     print("seq_len: ",seq_len)
    for di in range(seq_len):
        decoded_scores, y_last, (d_last,d_cell_last), attention_weights = decoder(Variable(torch.LongTensor([[y_last]])).to(device), \
                                                                                 encoder_outputs.to(device), \
                                                                                 d_last.to(device),\
                                                                                 d_cell_last.to(device))
        
        loss += criterion(decoded_scores.to(device), targets[:,di])   
    
    assert not np.isnan(loss.item()), 'Model diverged with loss = NaN'

    loss.backward()
    optimizer.step()
    
    if epoch % print_every == 0:
        print(f"At Epoch: %.1f"% epoch)
        print(f"Loss %.4f"% (loss/seq_len))
#     loss_vector.append(loss/seq_len)


In [ ]:
torch.save(encoder.state_dict(), "../data/seq2seq_encoder_uniref_201012.pt")
torch.save(decoder.state_dict(), "../data/seq2seq_decoder_uniref_201012.pt")